In [86]:
import pandas as pd
import numpy as np

from option_pricing import AmericanOption

In [87]:
date = pd.to_datetime('2013-06-03')

In [88]:
option_data = pd.read_csv('./data/option_data/2013-06-03options.csv')
stock_data = pd.read_csv('./data/stock_data/2013-06-03stocks.csv')

In [89]:
option_data['expiration'] = pd.to_datetime(option_data['expiration'])

In [90]:
option_data['time_to_expiry'] = (option_data['expiration'] - date).dt.days/252
option_data['mid_price'] = (option_data['ask']+option_data['bid'])/2

In [91]:
total_data = option_data.merge(stock_data[['symbol', 'close']],
                         left_on='underlying',
                         right_on='symbol')

In [92]:
total_data = total_data.loc[total_data['style']=='A']

In [93]:
total_data = total_data[['underlying', 'expiration', 'type', 'strike', 'mid_price', 'close', 'implied_volatility', 'time_to_expiry']]

In [94]:
total_data = total_data.iloc[:1000]

In [95]:
total_data['binomial_price'] = total_data.apply(lambda row: AmericanOption(S0=row.close, 
                                                                           K=row.strike,
                                                                           r=0.05,
                                                                           T=row.time_to_expiry,
                                                                           sigma=row.implied_volatility, 
                                                                           option_type=row.type
                                                                          ).price_binomial(), axis=1)

In [97]:
total_data['abs_err'] = np.abs(total_data['binomial_price'] - total_data['mid_price'])
total_data['signed_err'] = total_data['binomial_price'] - total_data['mid_price']
total_data['pct_err'] = total_data['abs_err'] / (total_data['mid_price'].abs() + 1e-8)

In [100]:
mae  = total_data['abs_err'].mean()
rmse = np.sqrt((total_data['signed_err']**2).mean())
mape = 100 * total_data['pct_err'].mean()
bias = total_data['signed_err'].mean()

In [102]:
print(f"MAE={mae:.4f}, RMSE={rmse:.4f}, MAPE={mape:.2f}%, Bias={bias:.4f}")

MAE=0.6423, RMSE=1.1894, MAPE=325400074.24%, Bias=0.5176


In [103]:
total_data['moneyness'] = total_data['close'] / total_data['strike']

total_data['moneyness_bin'] = pd.cut(total_data['moneyness'], bins=[0,0.9,0.97,1.03,1.1, np.inf],
                             labels=['deep OTM','OTM','ATM','ITM','deep ITM'])

total_data['mat_days'] = (total_data['time_to_expiry'] * 252).clip(lower=0).astype(int)

total_data['mat_bin'] = pd.cut(total_data['mat_days'], bins=[-1,7,30,90,365,9999],
                       labels=['<1w','1w-1m','1m-3m','3m-1y','>1y'])

In [104]:
total_data

,underlying,expiration,type,strike,mid_price,close,implied_volatility,time_to_expiry,binomial_price,abs_err,signed_err,pct_err,moneyness,moneyness_bin,mat_days,mat_bin
0,A,2013-06-22,call,28.0,17.475,45.510003,0.3659,0.075397,17.6154,0.1404,0.1404,0.008034,1.625357,deep ITM,19,1w-1m
1,A,2013-06-22,put,28.0,0.005,45.510003,0.4882,0.075397,0.0001,0.0049,-0.0049,0.979998,1.625357,deep ITM,19,1w-1m
2,A,2013-06-22,call,29.0,16.300,45.510003,0.3659,0.075397,16.6191,0.3191,0.3191,0.019577,1.569310,deep ITM,19,1w-1m
3,A,2013-06-22,put,29.0,0.010,45.510003,0.4882,0.075397,0.0004,0.0096,-0.0096,0.959999,1.569310,deep ITM,19,1w-1m
4,A,2013-06-22,call,30.0,15.250,45.510003,0.3659,0.075397,15.6229,0.3729,0.3729,0.024452,1.517000,deep ITM,19,1w-1m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,AAPL,2013-06-14,put,425.0,1.800,450.720009,0.3297,0.043651,3.1221,1.3221,1.3221,0.734500,1.060518,ITM,11,1w-1m
996,AAPL,2013-06-14,call,430.0,23.225,450.720009,0.3191,0.043651,25.5777,2.3527,2.3527,0.101300,1.048186,ITM,11,1w-1m
997,AAPL,2013-06-14,put,430.0,2.510,450.720009,0.3210,0.043651,3.9882,1.4782,1.4782,0.588924,1.048186,ITM,11,1w-1m
998,AAPL,2013-06-14,call,435.0,19.225,450.720009,0.3137,0.043651,21.7609,2.5359,2.5359,0.131906,1.036138,ITM,11,1w-1m


In [107]:
summary = total_data.groupby(['moneyness_bin','mat_bin']).agg(
    count=('abs_err','size'),
    mae=('abs_err','mean'),
    rmse=('signed_err', lambda x: np.sqrt((x**2).mean())),
    median_err=('signed_err','median')
).reset_index()

print(summary)

   moneyness_bin mat_bin  count       mae      rmse  median_err
0       deep OTM     <1w     28  0.628100  1.438605    -0.00415
1       deep OTM   1w-1m     36  0.104833  0.164708    -0.01945
2       deep OTM   1m-3m     58  0.152095  0.230777    -0.00760
3       deep OTM   3m-1y     96  0.373146  0.547321     0.01120
4       deep OTM     >1y     30  1.920603  2.376080     0.08860
5            OTM     <1w     18  0.386467  0.497973     0.23470
6            OTM   1w-1m     16  0.177175  0.232066     0.13605
7            OTM   1m-3m     30  0.235413  0.333219     0.10410
8            OTM   3m-1y     34  0.709041  0.998676     0.10870
9            OTM     >1y      6  2.822517  3.326225     1.34865
10           ATM     <1w     12  1.277617  1.405726     1.47305
11           ATM   1w-1m      6  0.240750  0.268521     0.24780
12           ATM   1m-3m     10  0.441240  0.530548     0.38835
13           ATM   3m-1y     14  1.039364  1.438692     0.75525
14           ATM     >1y      4  3.06672

/var/folders/jj/r0dn4_597_9fm9w3q_85t9140000gn/T/ipykernel_1409/3058419176.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = total_data.groupby(['moneyness_bin','mat_bin']).agg(
